# Final Project Exploration

This is our Team's Data Science Tool for Finance final project. In this project we attempt to replicate the results of Table 1 found in Monetary Tightening and US Bank Fragility in 2023: Mark-To-Market Losses and Uninsured Depositor Runs, written by Erica Xuewei Jiang, Gregor Matvos,Tomasz Piskorski, and Amit Seru.

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import warnings
import data_read
import load_assets
import calc_functions
import load_WRDS
import config
from pathlib import Path
# import Calc_table_statistic

BASE_DIR = config.BASE_DIR
data_dir = BASE_DIR / 'data' 
warnings.filterwarnings("ignore")

### Reports load

- We first import the necessary data from WRDS
- RCFD includes banks with both domestic and foreign branches
- RCON includes banks with only domestic branches
- These data series are divided into series 1 and series 2

In [ ]:
rcfd_data_1 = load_WRDS.load_RCFD_series_1(data_dir)
rcon_data_1 = load_WRDS.load_RCON_series_1(data_dir)
rcfd_data_2 = load_WRDS.load_RCFD_series_2(data_dir)
rcon_data_2 = load_WRDS.load_RCON_series_2(data_dir)

In [ ]:
rcfd_data_1.columns = [col.upper() for col in rcfd_data_1.columns]
rcon_data_1.columns = [col.upper() for col in rcon_data_1.columns]
rcfd_data_2.columns = [col.upper() for col in rcfd_data_2.columns]
rcon_data_2.columns = [col.upper() for col in rcon_data_2.columns]

In [ ]:
inital_date = '03/31/2022'
analysis_date = '03/31/2023'

## Data Cleaning

### Assets

- We then get assets from both the RCON and RCFD, and combine the dataframes to compute the total/gross asset at bank levels

In [ ]:

filtered_asset_level_0 = load_assets.clean_assets(rcfd_data_2,'RCFD2170',inital_date)
filtered_asset_level = load_assets.clean_assets(rcon_data_2,'RCON2170',inital_date)
filtered_asset_level.head()


#### Creating a dataframe for total/gross asset for each bank

In [ ]:
df_asset = pd.concat([filtered_asset_level_0, filtered_asset_level])
total_asset = df_asset['gross_asset'].sum()
df_asset.head()

### LOANS

- We then get assets from both the RCON and RCFD, and combine the dataframes to compute the total/gross asset at bank levels

In [ ]:
rcon_cols = ['RCONA564', 'RCONA565', 'RCONA566', 'RCONA567', 'RCONA568', 'RCONA569']
rcon_cols2 = ['RCONA570', 'RCONA571', 'RCONA572', 'RCONA573', 'RCONA574', 'RCONA575']
rcfd_cols = ['RCFDA570', 'RCFDA571', 'RCFDA572', 'RCFDA573', 'RCFDA574', 'RCFDA575']

# Residential Loans with repricing maturity (secured by first liens)
df_loans_first_lien_domestic = load_assets.clean_loans(rcon_data_1,rcon_cols,inital_date)

# Loans and Leases (not-secured by first liens)
df_loans_exc_first_lien = load_assets.clean_loans(rcfd_data_1,rcfd_cols,inital_date)
df_loans_exc_first_lien_domestic = load_assets.clean_loans(rcon_data_2,rcon_cols2,inital_date)

df_loans_first_lien_domestic.head()

In [ ]:
df_other_loan = pd.concat([df_loans_exc_first_lien_domestic, df_loans_exc_first_lien])
df_other_loan = df_other_loan.sort_index()
df_other_loan.head()

### RMBs

In [ ]:
rmbs_cols = ['RCFDA555', 'RCFDA556', 'RCFDA557', 'RCFDA558', 'RCFDA559', 'RCFDA560']
rmbs_dom_cols = ['RCONA555', 'RCONA556', 'RCONA557', 'RCONA558', 'RCONA559', 'RCONA560']
df_RMBS = load_assets.clean_loans(rcfd_data_1,rmbs_cols,inital_date)
df_RMBS_dom = load_assets.clean_loans(rcon_data_1,rmbs_dom_cols,inital_date) #Form 051 - Domestic

In [ ]:
df_RMBS_Final = pd.concat([df_RMBS_dom, df_RMBS])
df_RMBS_Final = df_RMBS_Final.sort_index()
df_RMBS_Final.head()

# Treausurys and Other (non-RMBs)

In [ ]:
non_RMBS_cols = ['RCFDA549', 'RCFDA550', 'RCFDA551', 'RCFDA552', 'RCFDA553', 'RCFDA554']
non_RMBS_dom_cols = ['RCONA549', 'RCONA550', 'RCONA551', 'RCONA552', 'RCONA553', 'RCONA554']

df_non_RMBS = load_assets.clean_loans(rcfd_data_2,non_RMBS_cols,inital_date)
df_non_RMBS_dom = load_assets.clean_loans(rcon_data_2,non_RMBS_dom_cols,inital_date)

In [ ]:
df_treasury_and_others = pd.concat([df_non_RMBS_dom, df_non_RMBS])
df_treasury_and_others = df_treasury_and_others.sort_index()
df_treasury_and_others.head()

In [ ]:
other_cols = ['RCFDA561', 'RCFDA562']
other_cols2 = ['RCONA561', 'RCONA562']
other_names = ['<3y', '>3y']

df_other_MBS =  load_assets.clean_others(rcfd_data_1,other_cols,other_names,inital_date)
df_other_MBS_domestic =  load_assets.clean_others(rcon_data_2,other_cols2,other_names,inital_date)
df_other_MBS_total = pd.concat([df_other_MBS, df_other_MBS_domestic])
df_other_MBS_total.head()

In [ ]:
other_leases = ['RCFDA247']
other_leases2 = ['RCONA247']
other_names2 = ['<1y']

df_other_loan_lease =  load_assets.clean_others(rcfd_data_1,other_leases,other_names2,inital_date)
df_other_loan_lease_domestic =  load_assets.clean_others(rcon_data_1,other_leases2,other_names2,inital_date)
df_total_other_loan_lease = pd.concat([df_other_loan_lease, df_other_loan_lease_domestic])
df_total_other_loan_lease.head()

# Asset Decomposition

- We caculated the total amount of core assets (RMBs, Treasuries, Residential Loans, Other Loans)
- We divided this by total amount of the asset (asset ratio), and calculate the ratios among the 4 categories
- We also calculated the ratio for other MBS and other leases, but these are not used in the subsequent analyses

In [ ]:
sum_asset = 0
for df in [df_RMBS_Final, df_loans_first_lien_domestic, df_treasury_and_others, df_other_loan]:
    total = df[['<3m', '3m-1y', '1y-3y', '3y-5y', '5y-15y', '>15y']].sum().sum()
    sum_asset += total

print('total assets:',"{:,.2f}".format(sum_asset))

In [ ]:
print('assets ratio:',"{:,.2%}".format(sum_asset/total_asset))

In [ ]:
RMBS = df_RMBS_Final[['<3m', '3m-1y', '1y-3y', '3y-5y', '5y-15y', '>15y']].sum().sum()
a = RMBS / total_asset
"{:,.2%}".format(a)

In [ ]:
df_loans = df_loans_first_lien_domestic[['<3m', '3m-1y', '1y-3y', '3y-5y', '5y-15y', '>15y']].sum().sum()
b = df_loans / total_asset
"{:,.2%}".format(b)

In [ ]:
df_treasury = df_treasury_and_others[['<3m', '3m-1y', '1y-3y', '3y-5y', '5y-15y', '>15y']].sum().sum()
c = df_treasury / total_asset
"{:,.2%}".format(c)

In [ ]:
df_other_ln = df_other_loan[['<3m', '3m-1y', '1y-3y', '3y-5y', '5y-15y', '>15y']].sum().sum()
d = df_other_ln / total_asset
"{:,.2%}".format(d)

In [ ]:
df_other_MBS = df_other_MBS_total[['<3y', '>3y']].sum().sum()
e = df_other_MBS  / total_asset
"{:,.2%}".format(e)

In [ ]:
df_total_other_ll = df_total_other_loan_lease['<1y'].sum()
f = df_total_other_ll / total_asset
"{:,.2%}".format(f)

In [ ]:
"{:,.2%}".format(a+b+c+d+e) #matches with the function above

## roughly the same as the security component (see Panel A)

In [ ]:
"{:,.2%}".format(a+e+c)

## roughly the same as the total loan component (see Panel A)

In [ ]:
"{:,.2%}".format(d+b)

# Table Analysis (Below are the contents for first column of table 1)

# 1. Aggregate Loss and Bank Loss (include each share)

###  Treasury prices

In [ ]:
path_combined = Path(data_dir) / "manual" / "combined_index_df"
path_treasury = Path(data_dir) / "manual" / "Treasury_index"
path_MBS = Path(data_dir) / "manual" / "MBS_ETF"

In [ ]:
combined_index_df = data_read.load_df(path_combined)
treasury_prices = combined_index_df[['iShares 0-1', 'iShares 1-3', 'sp 3-5', 'iShares 7-10', 'iShares 10-20', 'iShares 20+']]
treasury_prices = treasury_prices.resample('Q').first()
treasury_prices = treasury_prices.loc[inital_date:analysis_date]
treasury_prices

In [ ]:
df_SP_Treasury_bond_index = data_read.load_df(path_treasury,1)
df_SP_Treasury_bond_index = df_SP_Treasury_bond_index.resample('Q').first()
df_SP_Treasury_bond_index = df_SP_Treasury_bond_index.loc[inital_date:analysis_date]
df_SP_Treasury_bond_index


In [ ]:
df_iShare_MBS_ETF = data_read.load_df(path_MBS,1,csv=True)
df_iShare_MBS_ETF.index = pd.to_datetime(df_iShare_MBS_ETF.index)
df_iShare_MBS_ETF = df_iShare_MBS_ETF.resample('Q').first()
df_iShare_MBS_ETF.index.rename('date', inplace=True)
df_iShare_MBS_ETF = df_iShare_MBS_ETF.loc['2022-03-31':'2023-03-31']
df_iShare_MBS_ETF

In [ ]:
RMBS_Multiplier = calc_functions.RMBs_Multiplier(df_SP_Treasury_bond_index, df_iShare_MBS_ETF,inital_date,analysis_date)
RMBS_Multiplier

In [ ]:
#Create copies for each dataframe
df_RMBS_Final_1 = df_RMBS_Final.copy() 
df_loans_first_lien_domestic_1 = df_loans_first_lien_domestic.copy()
df_treasury_and_others_1 = df_treasury_and_others.copy()
df_other_loan_1 = df_other_loan.copy()

In [ ]:
bank_losses = calc_functions.report_losses(inital_date,analysis_date,df_RMBS_Final, df_loans_first_lien_domestic,\
                                            df_treasury_and_others, df_other_loan, treasury_prices, RMBS_Multiplier, df_asset)
bank_losses

In [ ]:
median_percentage = bank_losses[['Share RMBs', 'Share Treasury and Other', 
        'Share Residential Mortgage', 'Share Other Loan']].median()
median_percentage
#close to reported values, but the distribution looks good

In [ ]:
std_percentages =bank_losses[['Share RMBs', 'Share Treasury and Other', 
        'Share Residential Mortgage', 'Share Other Loan']].std()
std_percentages #this is slightly different 

## Total, median, and std of losses

In [ ]:
total_sum_loss = bank_losses['total_loss'].sum()
total_sum_loss

In [ ]:
median_bank_loss = bank_losses['total_loss'].median()
median_bank_loss 

In [ ]:
std_bank_loss = bank_losses['total_loss'].std()
std_bank_loss

## Checks

In [ ]:
core_asset = bank_losses['core_asset'].sum()
core_asset

In [ ]:
gross_asset = bank_losses['gross_asset'].sum()
gross_asset 

In [ ]:
core_asset / gross_asset

# 2. Loss / Asset

## Ratio Results

In [ ]:
median_loss_asset_ratio = bank_losses['loss/gross_asset'].median()
median_loss_asset_ratio

In [ ]:
average_loss_asset_ratio = bank_losses['loss/gross_asset'].mean()
average_loss_asset_ratio

In [ ]:
std_loss_asset_ratio = bank_losses['loss/gross_asset'].std()
std_loss_asset_ratio

# 3. Uninsured Deposit/MM Asset

- Uninsured Deposits are directly reported as RCON5597 in the call reports
- MM Asset: total assets in 2022:Q1 minus the mark-to-market value loss (value) 

In [ ]:
uninsured_deposit = rcon_data_1[['RSSD9001','RSSD9017', 'RSSD9999', 'RCON5597']]
uninsured_deposit = uninsured_deposit .rename(columns={
    'RSSD9001': 'bank_ID',
    'RSSD9017': 'bank_name',
    'RSSD9999': 'report_date',
    'RCON5597': 'uninsured_deposit'

})
uninsured_deposit = uninsured_deposit[uninsured_deposit['report_date'] == '03/31/2022']
uninsured_deposit['uninsured_deposit'].sum()/gross_asset #Similar to Panel A result of 37.4%

In [ ]:
uninsured_deposit['uninsured_deposit'].sum() #Roughly 9 trillion 

In [ ]:
def calculate_uninsured_deposit_mm_asset(uninsured_deposit, bank_losses):
    
    # Initialize an empty list to store the results
    results = []
    
    # Adjust the uninsured_deposit DataFrame to use both 'bank_name' and 'Bank_ID' as a multi-index for quick lookup
    uninsured_lookup = uninsured_deposit.set_index(['bank_name', 'bank_ID'])['uninsured_deposit'].to_dict()
    
    # Iterate over each row in bank_losses DataFrame
    for index, bank_loss_row in bank_losses.iterrows():
        bank_name = bank_loss_row['bank_name']
        bank_id = bank_loss_row['bank_ID']
        
        # Adjust the lookup to include 'Bank_ID'
        uninsured_deposit_value = uninsured_lookup.get((bank_name, bank_id), 0)
        
        # Calculate 'MM Asset' as the sum of 'total_loss' and 'gross_asset' (as defined in the paper)
        mm_asset = bank_loss_row['total_loss'] + bank_loss_row['gross_asset']
        
        # Calculate Uninsured Deposit/MM Asset ratio 
        if mm_asset > 0:
            uninsured_deposit_mm_asset_ratio = uninsured_deposit_value / mm_asset
        
        # Append to final dataframe
        results.append({
            'bank_name': bank_name,
            'bank_ID': bank_id, 
            'total_loss': bank_loss_row['total_loss'], 
            'total_asset': bank_loss_row['gross_asset'],
            'mm_asset': mm_asset,
            'uninsured_deposit': uninsured_deposit_value, 
            'Uninsured_Deposit_MM_Asset': uninsured_deposit_mm_asset_ratio
        })
    
    # Convert results list to DataFrame and sort by 'Bank_ID'
    results_df = pd.DataFrame(results).sort_values(by=['bank_name', 'bank_ID'])
    
    return results_df

un_mm_ratio = calculate_uninsured_deposit_mm_asset(uninsured_deposit, bank_losses)

## Checks

In [ ]:
un_mm_ratio['total_loss'].sum()

In [ ]:
un_mm_ratio['total_asset'].sum()

In [ ]:
un_mm_ratio['mm_asset'].sum()

## Ratio Results

In [ ]:
un_mm_ratio['Uninsured_Deposit_MM_Asset'].median()

In [ ]:
un_mm_ratio['Uninsured_Deposit_MM_Asset'].std()

# 4. Insured Deposit Coverage Ratio


#### we defined Insured Deposit Coverage Ratio as total amount of deposit accounts of:

- RCFDF049 - Deposit accounts (excluding retirement accounts) of 250000 or less
- RCFDF045 - Retirement deposit accounts of 250000 or less

In [ ]:
insured_deposit = rcon_data_1[['RSSD9001','RSSD9017', 'RSSD9999', 'RCONF049', 'RCONF045']] 
#RCFDF049 are Deposit accounts (excluding retirement accounts) of $250,000 or less
#RCFDF045 are Retirement deposit accounts of $250,000 or less

insured_deposit = insured_deposit.rename(columns={
    'RSSD9001': 'bank_ID',
    'RSSD9017': 'bank_name',
    'RSSD9999': 'report_date',
    'RCONF049': 'insured_deposit_1',
    'RCONF045': 'insured_deposit_2'
})
insured_deposit = insured_deposit[insured_deposit['report_date'] == '03/31/2022']
insured_deposit['insured_deposit_1'].sum()/total_asset + insured_deposit['insured_deposit_2'].sum()/total_asset

#THESE NUMBERS ARE SLIGHLY FAR FROM INSURED DEPOSIT IN PANEL A

In [ ]:
insured_deposit['insured_deposit'] = insured_deposit['insured_deposit_1'] + insured_deposit['insured_deposit_2']
insured_deposit 

In [ ]:
insured_deposit['insured_deposit'].sum()

In [ ]:
def insured_deposit_coverage_ratio(insured_deposit, uninsured_deposit, bank_losses):
    # Initialize an empty list to store the results
    results = []
    
    # Create dictionaries from insured and uninsured deposits for quick lookup
    insured_lookup = insured_deposit.set_index(['bank_name', 'bank_ID'])['insured_deposit'].to_dict()
    uninsured_lookup = uninsured_deposit.set_index(['bank_name', 'bank_ID'])['uninsured_deposit'].to_dict()
    
    # Iterate over each row in bank_losses DataFrame
    for index, bank_loss_row in bank_losses.iterrows():
        bank_name = bank_loss_row['bank_name']
        bank_id = bank_loss_row['bank_ID']
        
        # Retrieve insured and uninsured deposit values
        insured_deposit_value = insured_lookup.get((bank_name, bank_id), 0)
        uninsured_deposit_value = uninsured_lookup.get((bank_name, bank_id), 0)
        
        # Calculate mark-to-market asset value as the sum of 'total_asset' minus 'total_loss'
        mark_to_market_asset_value = bank_loss_row['gross_asset'] + bank_loss_row['total_loss']
        
        # Calculate the insured deposit coverage ratio
        if insured_deposit_value > 0:  # Prevent division by zero
            coverage_ratio = (mark_to_market_asset_value - uninsured_deposit_value - insured_deposit_value) / insured_deposit_value
        
        # Append the result
        results.append({
            'bank_name': bank_name,
            'bank_ID': bank_id,
            'mm_asset': mark_to_market_asset_value,
            'insured_deposit': insured_deposit_value,
            'uninsured_deposit': uninsured_deposit_value,
            'insured_deposit_coverage_ratio': coverage_ratio
        })
    
    # Convert results list to DataFrame
    results_df = pd.DataFrame(results)
    
    return results_df

coverage_ratios_df = insured_deposit_coverage_ratio(insured_deposit, uninsured_deposit, bank_losses)
coverage_ratios_df

## Checks

In [ ]:
coverage_ratios_df['insured_deposit'].sum()

In [ ]:
coverage_ratios_df['uninsured_deposit'].sum()

In [ ]:
coverage_ratios_df['mm_asset'].sum()

## Ratio Results

In [ ]:
coverage_ratios_df['insured_deposit_coverage_ratio'].median()

In [ ]:
coverage_ratios_df['insured_deposit_coverage_ratio'].std()

# Final Table Results

- For table 1, we used the above methods to aggregate the stastics across losses

In [ ]:
def final_statistic_table(bank_losses_assets, uninsured_deposit_mm_asset, insured_deposit_coverage, index_name = 'All Banks'):
    # Merge the DataFrames on bank_name and Bank_ID to include uninsured deposit/MM Asset ratios and insured deposit coverage ratios
    
    
    bank_count = len(bank_losses_assets.index)

    final_stats = pd.DataFrame({
        'Aggregate Loss': [f"{-round(bank_losses_assets['total_loss'].sum() / 1e9, 1)}T"],  # Convert to trillions
        'Bank Level Loss': [f"{-round(bank_losses_assets['total_loss'].median() / 1e3, 1)}M"],  # Convert to millions
        'Bank Level Loss Std': [f"{round(bank_losses_assets['total_loss'].std() / 1e6, 2)}B"],  # Std deviation for Bank Level Loss
        'Share RMBS': [round(bank_losses_assets['Share RMBs'].median() * 100, 1)],  # Median percentage
        'Share RMBS Std': [round(bank_losses_assets['Share RMBs'].std() * 100, 1)],  # Std deviation for Share RMBS
        'Share Treasury and Other': [round(bank_losses_assets['Share Treasury and Other'].median() * 100, 1)],  # Median percentage
        'Share Treasury and Other Std': [round(bank_losses_assets['Share Treasury and Other'].std() * 100, 1)],  # Std deviation
        'Share Residential Mortgage': [round(bank_losses_assets['Share Residential Mortgage'].median() * 100, 1)],  # Median percentage
        'Share Residential Mortgage Std': [round(bank_losses_assets['Share Residential Mortgage'].std() * 100, 1)],  # Std deviation
        'Share Other Loan': [round(bank_losses_assets['Share Other Loan'].median() * 100, 1)],  # Median percentage
        'Share Other Loan Std': [round(bank_losses_assets['Share Other Loan'].std() * 100, 1)],  # Std deviation
        'Loss/Asset': [round(bank_losses_assets['loss/gross_asset'].median() * 100, 1)],  # Median percentage
        'Loss/Asset Std': [round(bank_losses_assets['loss/gross_asset'].std() * 100, 1)],  # Std deviation
        'Uninsured Deposit/MM Asset': [round(uninsured_deposit_mm_asset['Uninsured_Deposit_MM_Asset'].median() * 100, 1)],  # Median percentage
        'Uninsured Deposit/MM Asset Std': [round(uninsured_deposit_mm_asset['Uninsured_Deposit_MM_Asset'].std() * 100, 1)],  # Std deviation
        'Insured Deposit Coverage Ratio': [round(insured_deposit_coverage['insured_deposit_coverage_ratio'].median() * 100, 1)],  # Median percentage
        'Insured Deposit Coverage Ratio Std': [round(insured_deposit_coverage['insured_deposit_coverage_ratio'].std() * 100, 1)],  # Std deviation
        'Number of Banks': [len(bank_losses_assets.index.unique())]  # Count of unique banks
    })

    # Rename index to 'All Banks'
    final_stats.index = [index_name]

    final_stats = final_stats.T
    
    return final_stats

In [ ]:
def GSIB_bank_id():
    #GSIB = [35301,93619,229913,398668,413208,451965,476810,480228,488318,
     #497404,541101,651448,688079,722777,812164,852218,934329,1225761,
     #1443266,1456501,2182786,2362458,2489805,2531991,3066025]
    GSIB = [852218, 480228, 476810, 413208, #JP Morgan, Bank of America, Citigroup, HSBC
      2980209, 2182786, 541101, 655839, 1015560, 229913,#Barclays, Goldman Sachs, BNY Mellon, CCB COMMUNITY BANK, ICBC, Mizuho
       1456501, 722777, 35301, 925411, 497404, 3212149, #Morgan Stanley, Santander, State Street, Sumitomo Mitsui, TD Bank, UBS
      451965] #wells fargo
    return GSIB

def large_ex_GSIB_bank_id(large):
    bank_id_large_ex_GSIB = []
    for bank_id in large['Bank_ID']:
       bank_id_large_ex_GSIB.append(bank_id)
    return bank_id_large_ex_GSIB

def small_bank_id(small):
    bank_id_small = []
    for bank_id in small['Bank_ID']:
       bank_id_small.append(bank_id)
    return bank_id_small

In [ ]:
GSIB = GSIB_bank_id() #list of GSIB bank IDs
df_asset_GSIB = df_asset[df_asset['Bank_ID'].isin(GSIB)] #total assets all GSIB banks
#Large non-GSIB Banks
df_asset_large_ex_GSIB = df_asset[(~df_asset['Bank_ID'].isin(GSIB)) & (df_asset['gross_asset']>1384000)] #total assets all large non-GSIB banks
large_ex_GSIB = large_ex_GSIB_bank_id(df_asset_large_ex_GSIB) #list of large non-GSIB bank IDs
#Small Banks
df_asset_small = df_asset[(~df_asset['Bank_ID'].isin(GSIB)) & (df_asset['gross_asset']<=1384000)] #total asset all small banks 
small = small_bank_id(df_asset_small)

In [ ]:
df_RMBS_GSIB = df_RMBS_Final[df_RMBS_Final['Bank_ID'].isin(GSIB)] #RMBS for GSIB banks
df_RMBS_large_ex_GSIB = df_RMBS_Final[df_RMBS_Final['Bank_ID'].isin(large_ex_GSIB)] #RMBS for large non-GSIB banks
df_RMBS_small = df_RMBS_Final[df_RMBS_Final['Bank_ID'].isin(small)]

In [ ]:
df_RMBS_Final = df_RMBS_Final #RMBS for all banks 
df_RMBS_GSIB = df_RMBS_Final[df_RMBS_Final['Bank_ID'].isin(GSIB)] #RMBS for GSIB banks
df_RMBS_large_ex_GSIB = df_RMBS_Final[df_RMBS_Final['Bank_ID'].isin(large_ex_GSIB)] #RMBS for large non-GSIB banks
df_RMBS_small = df_RMBS_Final[df_RMBS_Final['Bank_ID'].isin(small)] #RMBS for small banks

#Loans First Lien Domestic

df_loans_first_lien_domestic = df_loans_first_lien_domestic # loans first lien domestic for all banks
df_loans_first_lien_domestic_GSIB = df_loans_first_lien_domestic[df_loans_first_lien_domestic['Bank_ID'].isin(GSIB)] # loans first lien domestic for all GSIB banks
df_loans_first_lien_domestic_large_ex_GSIB = df_loans_first_lien_domestic[df_loans_first_lien_domestic['Bank_ID'].isin(large_ex_GSIB)] # loans first lien domestic for all large non-GSIB banks
df_loans_first_lien_domestic_small = df_loans_first_lien_domestic[df_loans_first_lien_domestic['Bank_ID'].isin(small)]

df_treasury_and_others = df_treasury_and_others #treasury and others all banks 
df_treasury_and_others_GSIB = df_treasury_and_others[df_treasury_and_others['Bank_ID'].isin(GSIB)] #treasury and others GSIB banks
df_treasury_and_others_large_ex_GSIB = df_treasury_and_others[df_treasury_and_others['Bank_ID'].isin(large_ex_GSIB)] #treasury and others large non-GSIB baanks 
df_treasury_and_others_small = df_treasury_and_others[df_treasury_and_others['Bank_ID'].isin(small)] #treasury and others small banks 

#Other Loan 

df_other_loan = df_other_loan #other loans for all banks 
df_other_loan_GSIB = df_other_loan[df_other_loan['Bank_ID'].isin(GSIB)] #other loans for all GSIB banks 
df_other_loan_large_ex_GSIB = df_other_loan[df_other_loan['Bank_ID'].isin(large_ex_GSIB)] #other loans for all large non-GSIB banks
df_other_loan_small = df_other_loan[df_other_loan['Bank_ID'].isin(small)] 

 #uninsured deposits
uninsured_deposit = uninsured_deposit #uninsured deposits for all banks
uninsured_deposit_GSIB = uninsured_deposit[uninsured_deposit['bank_ID'].isin(GSIB)] #uninsured deposits for GSIB banks
uninsured_deposit_large_ex_GSIB = uninsured_deposit[uninsured_deposit['bank_ID'].isin(large_ex_GSIB)] #uninsured deposits for large non-GSIB banks
uninsured_deposit_small = uninsured_deposit[uninsured_deposit['bank_ID'].isin(small)] #uninsured deposits for small banks

#insured deposits
insured_deposits = insured_deposit.copy() #insured deposits for all banks
insured_deposits_GSIB = insured_deposits[insured_deposits['bank_ID'].isin(GSIB)] #insured deposits for GSIB banks
insured_deposits_large_ex_GSIB = insured_deposits[insured_deposits['bank_ID'].isin(large_ex_GSIB)] #insured deposits for large non-GSIB banks
insured_deposits_small = insured_deposits[insured_deposits['bank_ID'].isin(small)]

In [ ]:
bank_losses = calc_functions.report_losses(inital_date,analysis_date,df_RMBS_Final, df_loans_first_lien_domestic,\
                                            df_treasury_and_others, df_other_loan, treasury_prices, RMBS_Multiplier, df_asset)

In [ ]:
bank_losses_assets = bank_losses.copy()
bank_losses_assets_small = calc_functions.report_losses(inital_date,analysis_date,df_RMBS_small, df_loans_first_lien_domestic_small, df_treasury_and_others_small, df_other_loan_small, treasury_prices, RMBS_Multiplier, df_asset_small)


In [ ]:
bank_losses_assets_large_ex_GSIB = calc_functions.report_losses(inital_date,analysis_date,df_RMBS_large_ex_GSIB, df_loans_first_lien_domestic_large_ex_GSIB, df_treasury_and_others_large_ex_GSIB, df_other_loan_large_ex_GSIB, treasury_prices, RMBS_Multiplier, df_asset_large_ex_GSIB)

In [ ]:
bank_losses_assets_GSIB = calc_functions.report_losses(inital_date,analysis_date,df_RMBS_GSIB, df_loans_first_lien_domestic_GSIB, df_treasury_and_others_GSIB, df_other_loan_GSIB, treasury_prices, RMBS_Multiplier, df_asset_GSIB)

In [ ]:
uninsured_deposit_mm_asset_small = calculate_uninsured_deposit_mm_asset(uninsured_deposit_small, bank_losses_assets_small)
insured_deposit_coverage_small = insured_deposit_coverage_ratio(insured_deposits_small, uninsured_deposit_small, bank_losses_assets_small)
uninsured_deposit_mm_asset_large_ex_GSIB = calculate_uninsured_deposit_mm_asset(uninsured_deposit_large_ex_GSIB, bank_losses_assets_large_ex_GSIB)
insured_deposit_coverage_large_ex_GSIB = insured_deposit_coverage_large_ex_GSIB = insured_deposit_coverage_ratio(insured_deposits_large_ex_GSIB, uninsured_deposit_large_ex_GSIB, bank_losses_assets_large_ex_GSIB)
uninsured_deposit_mm_asset_GSIB = calculate_uninsured_deposit_mm_asset(uninsured_deposit_GSIB, bank_losses_assets_GSIB)
insured_deposit_coverage_GSIB = insured_deposit_coverage_ratio(insured_deposits_GSIB, uninsured_deposit_GSIB, bank_losses_assets_GSIB)
    

In [ ]:
final_stats = final_statistic_table(bank_losses_assets, un_mm_ratio, coverage_ratios_df)
final_stats_small = final_statistic_table(bank_losses_assets_small, uninsured_deposit_mm_asset_small, insured_deposit_coverage_small, index_name = 'Small Banks')
final_stats_large_ex_GSIB = final_statistic_table(bank_losses_assets_large_ex_GSIB, uninsured_deposit_mm_asset_large_ex_GSIB, insured_deposit_coverage_large_ex_GSIB, index_name = 'Large Ex GSIB Banks')
final_stats_GSIB = final_statistic_table(bank_losses_assets_GSIB, uninsured_deposit_mm_asset_GSIB, insured_deposit_coverage_GSIB, index_name = 'GSIB Banks')

In [ ]:
table_1 = pd.concat([final_stats, final_stats_small, final_stats_large_ex_GSIB, final_stats_GSIB], axis=1)
table_1